## 1. 데이터 분석 연습하기
고객 3,500명에 대한 학습용 데이터를 이용하여 성별 예측 모형을 만든 후, 이를 평가용 데이터에 적용하여 얻은 2,482명 고객의 성별 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.
(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

### 1.1 데이터 탐색하기

In [1]:
# pandas 라이브러리 가져오기
import pandas as pd
# 주어진 데이터 파일을 모두 읽어서, 데이터 프레임 변수에 저장하기

x_train = pd.read_csv('bigData-main/x_train.csv', encoding='ms949')
x_test = pd.read_csv('bigData-main/x_test.csv', encoding='ms949')
y_train = pd.read_csv('bigData-main/y_train.csv', encoding='ms949')

In [2]:
print(x_train.head())

   cust_id      총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수  \
0        0  68282840  11264000  6860000.0      기타   강남점    19  3.894737   
1        1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000   
2        2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000   
3        3  16077620   4935000        NaN      기타   광주점    18  2.444444   
4        4  29050000  24000000        NaN      보석  본  점     2  1.500000   

     주말방문비율  구매주기  
0  0.527027    17  
1  0.000000     1  
2  0.000000     1  
3  0.318182    16  
4  0.000000    85  


In [3]:
print(x_test.head())

   cust_id       총구매액     최대구매액        환불금액 주구매상품 주구매지점  내점일수    내점당구매건수  \
0     3500   70900400  22000000   4050000.0    골프  부산본점    13   1.461538   
1     3501  310533100  38558000  48034700.0   농산물   잠실점    90   2.433333   
2     3502  305264140  14825000  30521000.0  가공식품  본  점   101  14.623762   
3     3503    7594080   5225000         NaN  주방용품  부산본점     5   2.000000   
4     3504    1795790   1411200         NaN   수산품  청량리점     3   2.666667   

     주말방문비율  구매주기  
0  0.789474    26  
1  0.369863     3  
2  0.083277     3  
3  0.000000    47  
4  0.125000     8  


In [4]:
print(y_train.head())

   cust_id  gender
0        0       0
1        1       0
2        2       1
3        3       1
4        4       0


#### 말 줄임표로 확인이 어려운 데이터를 확인하는 방법
(1) T, transpose() 함수 사용해서 열, 행 방향 바꾸기  
(2) 전체 칼럼을 출력하는 옵션 사용하기
 - pd.options.display.max_columns = None

In [5]:
pd.options.display.max_columns = None
print(x_train.head())

   cust_id      총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수  \
0        0  68282840  11264000  6860000.0      기타   강남점    19  3.894737   
1        1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000   
2        2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000   
3        3  16077620   4935000        NaN      기타   광주점    18  2.444444   
4        4  29050000  24000000        NaN      보석  본  점     2  1.500000   

     주말방문비율  구매주기  
0  0.527027    17  
1  0.000000     1  
2  0.000000     1  
3  0.318182    16  
4  0.000000    85  


In [6]:
# 행 / 열 확인하기
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(3500, 10)
(2482, 10)
(3500, 2)


In [7]:
# 요약 정보 확인하기
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB
None


In [8]:
# 기초 통계량 확인하기
print(x_train.describe().T)

          count          mean           std         min           25%  \
cust_id  3500.0  1.749500e+03  1.010507e+03         0.0  8.747500e+02   
총구매액     3500.0  9.191925e+07  1.635065e+08 -52421520.0  4.747050e+06   
최대구매액    3500.0  1.966424e+07  3.199235e+07  -2992000.0  2.875000e+06   
환불금액     1205.0  2.407822e+07  4.746453e+07      5600.0  2.259000e+06   
내점일수     3500.0  1.925371e+01  2.717494e+01         1.0  2.000000e+00   
내점당구매건수  3500.0  2.834963e+00  1.912368e+00         1.0  1.666667e+00   
주말방문비율   3500.0  3.072463e-01  2.897516e-01         0.0  2.729090e-02   
구매주기     3500.0  2.095829e+01  2.474868e+01         0.0  4.000000e+00   

                  50%           75%           max  
cust_id  1.749500e+03  2.624250e+03  3.499000e+03  
총구매액     2.822270e+07  1.065079e+08  2.323180e+09  
최대구매액    9.837000e+06  2.296250e+07  7.066290e+08  
환불금액     7.392000e+06  2.412000e+07  5.637530e+08  
내점일수     8.000000e+00  2.500000e+01  2.850000e+02  
내점당구매건수  2.333333e+00  3.37500

### 1.2 전처리하기

In [9]:
# 테스트 데이터의 cust_id 값은 x_test_cust_id 변수에 저장하기
# 분석과정에는 불필요하나 추후 제출에 필요하다
x_test_cust_id = x_test['cust_id']

In [10]:
# cust_id 칼럼을 삭제하기
x_train = x_train.drop(columns=['cust_id'])
y_train = y_train.drop(columns=['cust_id'])
x_test = x_test.drop(columns=['cust_id'])

# 칼럼이 삭제된 상위 5개 행을 확인하기
print(x_train.head())
print(y_train.head())

       총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수    주말방문비율  구매주기
0  68282840  11264000  6860000.0      기타   강남점    19  3.894737  0.527027    17
1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000  0.000000     1
2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000  0.000000     1
3  16077620   4935000        NaN      기타   광주점    18  2.444444  0.318182    16
4  29050000  24000000        NaN      보석  본  점     2  1.500000  0.000000    85
   gender
0       0
1       0
2       1
3       1
4       0


In [11]:
# 결측치 처리하기
print(x_train.isnull().sum())

총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64


In [12]:
# '환불금액' 칼럼의 결측치는 0으로 대치하기
x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)
# 결측치가 조치되었는지, '환불금액' 칼럼의 결측치 개수를 확인하기
print(x_train['환불금액'].isnull().sum())
print(x_test['환불금액'].isnull().sum())

0
0


In [13]:
# 범주형 변수를 인코딩하기

# '주구매상품' 칼럼에서 중복을 제외한 값들을 확인하기
print(x_train['주구매상품'].unique())
# '주구매상품' 칼럼에서 중복을 제외한 값들의 개수 세기
print(x_train['주구매상품'].unique().size)
# '주구매지점' 칼럼에서 중복을 제외한 값들을 확인하기
print(x_train['주구매지점'].unique())
# '주구매지점' 칼럼에서 중복을 제외한 값들의 개수 세기
print(x_train['주구매지점'].unique().size)


['기타' '스포츠' '남성 캐주얼' '보석' '디자이너' '시티웨어' '명품' '농산물' '화장품' '골프' '구두' '가공식품'
 '수산품' '아동' '차/커피' '캐주얼' '섬유잡화' '육류' '축산가공' '젓갈/반찬' '액세서리' '피혁잡화' '일용잡화'
 '주방가전' '주방용품' '건강식품' '가구' '주류' '모피/피혁' '남성 트랜디' '셔츠' '남성정장' '생활잡화'
 '트래디셔널' '란제리/내의' '커리어' '침구/수예' '대형가전' '통신/컴퓨터' '식기' '소형가전' '악기']
42
['강남점' '잠실점' '관악점' '광주점' '본  점' '일산점' '대전점' '부산본점' '분당점' '영등포점' '미아점'
 '청량리점' '안양점' '부평점' '동래점' '포항점' '노원점' '창원점' '센텀시티점' '인천점' '대구점' '전주점'
 '울산점' '상인점']
24


In [14]:
from sklearn.preprocessing import LabelEncoder
# 라벨 인코딩 수행을 위한 encoder 함수 만들기
encoder = LabelEncoder()

In [15]:
# '주구매상품'에 대해 라벨 인코딩을 수행한 후, '주구매상품'칼럼으로 다시 저장하기
x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
# 라벨인코딩 결과를 확인하기 위해, 상위 10개 행을 확인하기
print(x_train['주구매상품'].head(10))

0     5
1    21
2     6
3     5
4    15
5    11
6    22
7    13
8     5
9     9
Name: 주구매상품, dtype: int32


In [16]:
# '주구매상품' 칼럼에 대한 라벨 인코딩의 변환 순서 확인하기
print(encoder.classes_)

# 테스트 데이터로 라벨 인코딩 수행하기
x_test['주구매상품'] = encoder.fit_transform(x_test['주구매상품'])
print(encoder.classes_)

['가공식품' '가구' '건강식품' '골프' '구두' '기타' '남성 캐주얼' '남성 트랜디' '남성정장' '농산물' '대형가전'
 '디자이너' '란제리/내의' '명품' '모피/피혁' '보석' '생활잡화' '섬유잡화' '셔츠' '소형가전' '수산품' '스포츠'
 '시티웨어' '식기' '아동' '악기' '액세서리' '육류' '일용잡화' '젓갈/반찬' '주류' '주방가전' '주방용품'
 '차/커피' '축산가공' '침구/수예' '캐주얼' '커리어' '통신/컴퓨터' '트래디셔널' '피혁잡화' '화장품']
['가공식품' '가구' '건강식품' '골프' '구두' '기타' '남성 캐주얼' '남성 트랜디' '남성정장' '농산물' '대형가전'
 '디자이너' '란제리/내의' '명품' '모피/피혁' '보석' '생활잡화' '섬유잡화' '셔츠' '수산품' '스포츠' '시티웨어'
 '식기' '아동' '악기' '액세서리' '육류' '일용잡화' '젓갈/반찬' '주류' '주방가전' '주방용품' '차/커피'
 '축산가공' '침구/수예' '캐주얼' '커리어' '통신/컴퓨터' '트래디셔널' '피혁잡화' '화장품']


In [17]:
# x_train컬럼이 더 많은 경우
if len(x_train.columns) >= len(x_test.columns) :
    x_train = x_train[x_test.columns]
else : # x_test컬럼이 더 많은 경우
    x_test = x_test[x_train.columns]

In [18]:
######### 물어볼 내용 ###########
# fit_transform 은 처음에만하고
# test 데이터에는 transform 만 해야하나?
print(x_train['주구매상품'].value_counts().sort_index())
print(x_test['주구매상품'].value_counts().sort_index())

0     546
1      10
2      47
3      82
4      54
5     595
6      55
7       2
8      22
9     339
10      8
11    193
12      8
13    100
14     57
15      3
16     15
17     98
18     30
19      2
20    153
21     69
22    213
23      7
24     40
25      2
26      5
27     57
28     64
29     29
30     14
31     26
32     32
33     44
34     35
35      4
36    101
37      9
38      3
39     23
40     40
41    264
Name: 주구매상품, dtype: int64
0     395
1       7
2      36
3      47
4      44
5     465
6      46
7       4
8      25
9     235
10      6
11    123
12     10
13     60
14     47
15      1
16      8
17     72
18     25
19     86
20     43
21    168
22      8
23     24
24      1
25      7
26     38
27     30
28     18
29     10
30     10
31     21
32     55
33     28
34      3
35     65
36      6
37      3
38      9
39     16
40    177
Name: 주구매상품, dtype: int64


In [19]:
# '주구매지점'에 대해 라벨 인코딩을 수행한 후, '주구매지점' 칼럼으로 다시 저장하기
x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])

# 라벨인코딩 결과를 확인하기 위해, 상위 10개 행을 확인하기
print(x_train['주구매지점'].head(10))

0     0
1    19
2     1
3     2
4     8
5    18
6     0
7     8
8     8
9     5
Name: 주구매지점, dtype: int32


In [20]:
# '주구매지점' 칼럼에 대한 라벨 인코딩의 변환 순서 확인하기
print(encoder.classes_)
# 테스트 데이터의 '주구매지점' 칼럼도 라벨 인코딩 수행하기
x_test['주구매지점'] = encoder.fit_transform(x_test['주구매지점'])
print(encoder.classes_)


['강남점' '관악점' '광주점' '노원점' '대구점' '대전점' '동래점' '미아점' '본  점' '부산본점' '부평점' '분당점'
 '상인점' '센텀시티점' '안양점' '영등포점' '울산점' '인천점' '일산점' '잠실점' '전주점' '창원점' '청량리점'
 '포항점']
['강남점' '관악점' '광주점' '노원점' '대구점' '대전점' '동래점' '미아점' '본  점' '부산본점' '부평점' '분당점'
 '상인점' '센텀시티점' '안양점' '영등포점' '울산점' '인천점' '일산점' '잠실점' '전주점' '창원점' '청량리점'
 '포항점']


In [21]:
# '환불금액' 칼럼이 0보다 큰지에 대한 조건을 condition 변수에 저장하기
condition = x_train['환불금액'] > 0

In [22]:
# condition 조건에 맞으면 '환불금액_new' 칼럼을 1로 설정하기
x_train.loc[condition,'환불금액_new'] = 1
# condition 조건이 맞지 않으면, '환불금액_new' 칼럼을 0으로 설정하기
x_train.loc[~condition,'환불금액_new'] = 0

In [23]:
# '환불금액', '환불금액_new' 칼럼을 비교하기
print(x_train[['환불금액', '환불금액_new']])

           환불금액  환불금액_new
0     6860000.0       1.0
1      300000.0       1.0
2           0.0       0.0
3           0.0       0.0
4           0.0       0.0
...         ...       ...
3495        0.0       0.0
3496  6049600.0       1.0
3497        0.0       0.0
3498        0.0       0.0
3499  5973000.0       1.0

[3500 rows x 2 columns]


In [24]:
# '환불금액'칼럼의 삭제 내용은 바로 데이터 세트에 반영하기
x_train = x_train.drop(columns=['환불금액'])

In [25]:
# 테스트 데이터 세트에서도 '환불금액_new'라는 파생변수 만들기(~는 not의 의미)
x_test.loc[condition, '환불금액_new'] = 1
x_test.loc[~condition, '환불금액_new'] = 0

# '환불금액' 칼럼의 삭제 내용은 바로 데이터 세트에 반영하기
x_test = x_test.drop(columns=['환불금액'])

In [26]:
# 크기변환 전, x_train 세트의 기초 통계량 확인하기
print(x_train.describe().T)

           count          mean           std         min           25%  \
총구매액      3500.0  9.191925e+07  1.635065e+08 -52421520.0  4.747050e+06   
최대구매액     3500.0  1.966424e+07  3.199235e+07  -2992000.0  2.875000e+06   
주구매상품     3500.0  1.461200e+01  1.301995e+01         0.0  5.000000e+00   
주구매지점     3500.0  1.073429e+01  5.636480e+00         0.0  8.000000e+00   
내점일수      3500.0  1.925371e+01  2.717494e+01         1.0  2.000000e+00   
내점당구매건수   3500.0  2.834963e+00  1.912368e+00         1.0  1.666667e+00   
주말방문비율    3500.0  3.072463e-01  2.897516e-01         0.0  2.729090e-02   
구매주기      3500.0  2.095829e+01  2.474868e+01         0.0  4.000000e+00   
환불금액_new  3500.0  3.442857e-01  4.752027e-01         0.0  0.000000e+00   

                   50%           75%           max  
총구매액      2.822270e+07  1.065079e+08  2.323180e+09  
최대구매액     9.837000e+06  2.296250e+07  7.066290e+08  
주구매상품     9.000000e+00  2.200000e+01  4.100000e+01  
주구매지점     9.000000e+00  1.500000e+01  2.300000e

In [27]:
# sklearn 패키지의 preprocessing 모듈에서 StandardScaler 함수를 가져오기
from sklearn.preprocessing import StandardScaler
# 표준화 크기 변환을 수행하기 위한 scaler 객체를 만들기
scaler = StandardScaler()
# scaler 객체로 표준화 크기변환을 수행하고, x_train의 칼럼명을 사용하기
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)

In [28]:
# 테스트 데이터는 transform() 함수만으로 값을 변환
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)

In [29]:
# 크기 변환 후, x_train 세트의 기초 통계량 확인하기
print(x_train.describe().T)

           count          mean       std       min       25%       50%  \
총구매액      3500.0 -4.409171e-17  1.000143 -0.882909 -0.533218 -0.389621   
최대구매액     3500.0 -4.838986e-17  1.000143 -0.708278 -0.524864 -0.307219   
주구매상품     3500.0 -6.236281e-17  1.000143 -1.122438 -0.738357 -0.431093   
주구매지점     3500.0 -8.333017e-17  1.000143 -1.904703 -0.485175 -0.307733   
내점일수      3500.0  2.518303e-16  1.000143 -0.671807 -0.635003 -0.414180   
내점당구매건수   3500.0 -2.288170e-16  1.000143 -0.959661 -0.611003 -0.262346   
주말방문비율    3500.0  9.192647e-17  1.000143 -1.060530 -0.966329 -0.175472   
구매주기      3500.0 -1.220294e-16  1.000143 -0.846966 -0.685318 -0.321610   
환불금액_new  3500.0  3.445498e-16  1.000143 -0.724606 -0.724606 -0.724606   

               75%        max  
총구매액      0.089237  13.648260  
최대구매액     0.103110  21.475852  
주구매상품     0.567518   2.027026  
주구매지점     0.756913   2.176441  
내점일수      0.211486   9.780490  
내점당구매건수   0.282432  10.066639  
주말방문비율    0.489224   2.391196  
구매주

In [30]:
# '총구매액', '최대구매액', '환불금액_new' 칼럼 간의 상관관계 구하기
print(x_train[['총구매액', '최대구매액', '환불금액_new']].corr())

              총구매액     최대구매액  환불금액_new
총구매액      1.000000  0.700080  0.403357
최대구매액     0.700080  1.000000  0.330687
환불금액_new  0.403357  0.330687  1.000000


In [31]:
# x_train 세트에서 '최대구매액' 칼럼을 삭제한 후, x_train에 저장하기
x_train = x_train.drop(columns=['최대구매액'])
# x_test 세트에서 '최대구매액' 칼럼을 삭제한 후, x_test에 저장하기
x_test = x_test.drop(columns=['최대구매액'])

### 1.3 학습하고 평가하기

In [32]:
# sklearn 패키지의 tree 모듈에서 DecisionTreeClassifier 함수를 가져오기
from sklearn.tree import DecisionTreeClassifier
# 의사결정나무 방식으로 수행되는 model 객체를 만들기
model = DecisionTreeClassifier()
# x_train, y_train 세트로 model을 학습시키기
model.fit(x_train, y_train)
# 학습된 model을 활용하여 테스트 데이터의 종속변수를 예측하기
y_test_predicted = model.predict(x_test)
# 예측한 결과를 출력해보기
print(pd.DataFrame(y_test_predicted).head(3))
print(type(y_test_predicted))
print(y_test_predicted)

   0
0  1
1  0
2  1
<class 'numpy.ndarray'>
[1 0 1 ... 0 1 1]


In [33]:
# 하이퍼파라미터 튜닝
# 괄호 안에 하이퍼 파라미터를 입력하여 모델 객체를 생성하기
model = DecisionTreeClassifier(max_depth=10, criterion='entropy', random_state=10)
# x_train, y_train 세트로 model을 학습시키기
model.fit(x_train, y_train)
# 학습된 model을 활용하여 테스트 데이터의 종속변수를 예측하기
y_test_predicted = model.predict(x_test)
# 예측한 결과를 출력해보기
print(pd.DataFrame(y_test_predicted).head(3))

   0
0  1
1  0
2  1


### GridSearchCV() 함수로 하이퍼파라미터 튜닝하기
- 하이퍼파라미터의 최적 조합을 찾을 때 사용함

In [34]:
# 하이퍼파라미터 튜닝을 위한 GridSearchCV 함수 가져오기
from sklearn.model_selection import GridSearchCV
# 병렬처리를 위한 Pipeline 함수 가져오기
from sklearn.pipeline import Pipeline

In [35]:
# 하이퍼 파라미터들과 튜닝할 값들을 작성하기
parameters = {'model__max_depth' : [3,4,5,6],
             'model__criterion' : ['gini', 'entropy']}

In [36]:
# 데이터 스케일링과 의사결정나무 분류작업을 병렬처리하도록 모델 만들기
pipeline_model = Pipeline([('scaler', StandardScaler()), ('model', DecisionTreeClassifier())])

In [37]:
grid_model = GridSearchCV(pipeline_model, param_grid=parameters, cv=3) # cv : 교차검증 수행횟수

In [38]:
print(help(GridSearchCV))

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "score_samples", "predict", "predict_proba",
 |  "decision_function", "transform" and "inverse_transform" if they are
 |  implemented in the estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object
 |      This is assumed to implement the scikit-learn estimator interface.
 |      Either est

In [39]:
print(grid_model.fit(x_train, y_train))

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', DecisionTreeClassifier())]),
             param_grid={'model__criterion': ['gini', 'entropy'],
                         'model__max_depth': [3, 4, 5, 6]})


In [40]:
best_model = grid_model.best_estimator_

In [41]:
y_test_predicted = best_model.predict(x_test)
print(pd.DataFrame(y_test_predicted))

      0
0     0
1     0
2     0
3     0
4     0
...  ..
2477  1
2478  0
2479  0
2480  0
2481  1

[2482 rows x 1 columns]


### GridSearchCV 내용 끝  
### 결과 예측하기

In [42]:
# model을 통해 x_test에 맞는 종속변수 확률을 구하기
y_test_proba = model.predict_proba(x_test)
# 종속변수의 0, 1에 대한 각 확률을 확인하기
print(pd.DataFrame(y_test_proba).head())

          0         1
0  0.200000  0.800000
1  1.000000  0.000000
2  0.000000  1.000000
3  0.548837  0.451163
4  0.548837  0.451163


In [43]:
# 성별이 남성인 확률값을 데이터 프레임 형태로 출력하기 
print(pd.DataFrame(y_test_proba)[1])

0       0.800000
1       0.000000
2       1.000000
3       0.451163
4       0.451163
          ...   
2477    0.462312
2478    0.451163
2479    0.338028
2480    0.000000
2481    0.512821
Name: 1, Length: 2482, dtype: float64


In [44]:
# 위 결과를 result 변수에 저장하기 
result = pd.DataFrame(y_test_proba)[1]

In [45]:
# x_train에 대한 종속 변수를 예측하기
y_train_predicted = model.predict(x_train)
# roc 평가지표를 계산하기 위한 함수를 가져오기
from sklearn.metrics import roc_auc_score
# 학습데이터에 대한 y_train, y_train_predicted의 ROC 결과 확인하기
print(roc_auc_score(y_train, y_train_predicted))

0.6896432468240978


### 테스트데이터로 모델 평가하기
- 주어진 학습데이터를 분리하여 검증용 데이터로 평가하자

In [46]:
# 데이터 분리하기 위한 train_test_split 함수를 가져오기
from sklearn.model_selection import train_test_split
# 학습용과 검증용을 8:2로 분리하기
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)
# 분리된 데이터의 행/열 구조를 확인하기
print(X_TRAIN.shape)
print(X_TEST.shape)
print(Y_TRAIN.shape)
print(Y_TEST.shape)

(2800, 8)
(700, 8)
(2800, 1)
(700, 1)


In [47]:
# 의사결정나무 모델을 생성하기
model = DecisionTreeClassifier(max_depth=10, criterion='entropy', random_state=10)
# 분리된 X_TRAIN, Y_TRAIN으로 모델을 학습시키기
model.fit(X_TRAIN, Y_TRAIN)
# 학습한 모델을 통해 X_TEST 세트에 대응되는 Y_TEST 세트를 예측하기
Y_TEST_PREDICTED = model.predict(X_TEST)

In [48]:
# sklearn 패키지의 metrics 모듈에서 roc_auc_score 함수를 가져오기
from sklearn.metrics import roc_auc_score
# roc 평가지표 값을 확인하기
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))

0.5771332492643968


### 1.4 결과 제출하기

In [49]:
# x_test_cust_id 변수와 result 변수를 세로 방향으로 붙이기
pd.concat([x_test_cust_id, result], axis=1)

,cust_id,1
0,3500,0.800000
1,3501,0.000000
2,3502,1.000000
3,3503,0.451163
4,3504,0.451163
...,...,...
2477,5977,0.462312
2478,5978,0.451163
2479,5979,0.338028
2480,5980,0.000000


In [50]:
# '1' 칼럼명을 'gender' 칼럼명으로 변환하여 다시 결과를 확인하기
pd.concat([x_test_cust_id, result], axis=1).rename(columns={1:'gender'})

,cust_id,gender
0,3500,0.800000
1,3501,0.000000
2,3502,1.000000
3,3503,0.451163
4,3504,0.451163
...,...,...
2477,5977,0.462312
2478,5978,0.451163
2479,5979,0.338028
2480,5980,0.000000


In [51]:
# 앞의 출력 결과를 final 변수에 저장하기
final = pd.concat([x_test_cust_id, result], axis=1).rename(columns={1:'gender'})
# final 변수를 data 디렉터리 하위에 12345.csv 이름으로 저장하기
final.to_csv('data/12345.csv', index=False)

### 제출된 파일이 정상적으로 저장되었는지 다시 확인하자

In [52]:
# data 디렉터리 하위의 12345.csv 파일을 final 변수에 저장하기
final = pd.read_csv('data/12345.csv')
# final 변수의 데이터 확인하기
print(final)

      cust_id    gender
0        3500  0.800000
1        3501  0.000000
2        3502  1.000000
3        3503  0.451163
4        3504  0.451163
...       ...       ...
2477     5977  0.462312
2478     5978  0.451163
2479     5979  0.338028
2480     5980  0.000000
2481     5981  0.512821

[2482 rows x 2 columns]


In [53]:
# 최종 제출 코드
# 라이브러리와 파일 읽기
import pandas as pd
x_train = pd.read_csv('bigData-main/x_train.csv', encoding='ms949')
x_test = pd.read_csv('bigData-main/x_test.csv', encoding='ms949')
y_train = pd.read_csv('bigData-main/y_train.csv', encoding='ms949')

# 테스트 데이터의 cust_id 저장하기
x_test_cust_id = x_test['cust_id']

# cust_id 컬럼 삭제하기
x_train = x_train.drop(columns=['cust_id'])
x_test = x_test.drop(columns=['cust_id'])
y_train = y_train.drop(columns=['cust_id'])

# 결측치 처리하기 
x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)

# 라벨인코딩 수행하기
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
x_test['주구매상품'] = encoder.transform(x_test['주구매상품'])
x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])
x_test['주구매지점'] = encoder.transform(x_test['주구매지점'])

# 파생변수 만들기
condition = x_train['환불금액'] > 0
x_train.loc[condition, '환불금액_new'] = 1
x_train.loc[~condition, '환불금액_new'] = 0
x_train = x_train.drop(columns=['환불금액'])
x_test.loc[condition, '환불금액_new'] = 1
x_test.loc[~condition, '환불금액_new'] = 0
x_test = x_test.drop(columns=['환불금액'])

# 데이터 스케일링 : 표준화 크기로 변환하기
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)

# 불필요한 칼럼 삭제하기
x_train = x_train.drop(columns=['최대구매액'])
x_test = x_test.drop(columns=['최대구매액'])

# 학습용/검증용 데이터로 분리하기
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

# 모델 학습 및 평가하기
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=10, criterion='entropy', random_state=10)
model.fit(X_TRAIN, Y_TRAIN)
y_test_proba = model.predict_proba(x_test)

# 결과 제출하기
result = pd.DataFrame(y_test_proba)[1]
final = pd.concat([x_test_cust_id, result], axis=1).rename(columns={1:'gender'})
print(final)
final.to_csv('data/12345.csv', index=False)

      cust_id    gender
0        3500  0.000000
1        3501  0.194215
2        3502  0.000000
3        3503  0.490909
4        3504  1.000000
...       ...       ...
2477     5977  0.393443
2478     5978  0.533632
2479     5979  0.000000
2480     5980  0.285714
2481     5981  0.000000

[2482 rows x 2 columns]


## 2. 분류모델 수행하기
고객 891명에 대한 학습용 데이터(titanic_x_train.csv, titanic_y_train.csv)를 이용하여 생존여부를 예측하는 모형을 만듭니다. 이후 예측 모형을 평가용 데이터(titanic_x_test.csv)에 적용하여 418명 승객의 생존 여부 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.
(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

### 2.1 데이터 탐색하기

In [54]:
# pandas 라이브러리 가져오기
import pandas as pd 

# 주어진 데이터 파일을 모두 읽어서, 각 데이터 프레임 변수에 저장하기
x_train = pd.read_csv('bigData-main/titanic_x_train.csv', encoding='cp949')
x_test = pd.read_csv('bigData-main/titanic_x_test.csv', encoding='cp949')
y_train = pd.read_csv('bigData-main/titanic_y_train.csv')

In [55]:
# x_train, x_test의 상위 1개 행을 확인하기
print(x_train.head(1).T)
print(x_test.head(1).T)

                                   0
PassengerId                        1
티켓등급                               3
승객이름         Braund, Mr. Owen Harris
성별                              male
나이                              22.0
형제자매배우자수                           1
부모자식수                              0
티켓번호                       A/5 21171
운임요금                            7.25
객실번호                             NaN
선착장                                S
                            0
PassengerId               892
티켓등급                        3
승객이름         Kelly, Mr. James
성별                       male
나이                       34.5
형제자매배우자수                    0
부모자식수                       0
티켓번호                   330911
운임요금                   7.8292
객실번호                      NaN
선착장                         Q


In [56]:
# y_train의 상위 5개 행을 가져오기
print(y_train.head(5))

   PassengerId  Survived
0            1         0
1            2         1
2            3         1
3            4         1
4            5         0


In [57]:
# 각 데이터 세트의 행과 열의 개수를 확인하기
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(891, 11)
(418, 11)
(891, 2)


In [58]:
# x_train 세트의 요약정보 확인하기
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   티켓등급         891 non-null    int64  
 2   승객이름         891 non-null    object 
 3   성별           891 non-null    object 
 4   나이           714 non-null    float64
 5   형제자매배우자수     891 non-null    int64  
 6   부모자식수        891 non-null    int64  
 7   티켓번호         891 non-null    object 
 8   운임요금         891 non-null    float64
 9   객실번호         204 non-null    object 
 10  선착장          889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB
None


In [59]:
# '성별' 칼럼의 중복 제거한 값과 개수 확인하기
print(x_train['성별'].unique().size)
print(x_train['성별'].unique())

2
['male' 'female']


In [60]:
# '선착장' 칼럼의 중복 제거한 값과 개수 확인하기
print(x_train['선착장'].unique().size)
print(x_train['선착장'].unique())


4
['S' 'C' 'Q' nan]


In [61]:
# 나머지 3개의 칼럼의 중복 제거한 값의 개수 확인하기
print(x_train['승객이름'].unique().size)
print(x_train['티켓번호'].unique().size)
print(x_train['객실번호'].unique().size)

891
681
148


In [62]:
# x_train의 기초 통계량을 확인하고, 가독성을 위해 행/열 바꿔서 출력하기
print(x_train.describe().T)

             count        mean         std   min       25%       50%    75%  \
PassengerId  891.0  446.000000  257.353842  1.00  223.5000  446.0000  668.5   
티켓등급         891.0    2.308642    0.836071  1.00    2.0000    3.0000    3.0   
나이           714.0   29.699118   14.526497  0.42   20.1250   28.0000   38.0   
형제자매배우자수     891.0    0.523008    1.102743  0.00    0.0000    0.0000    1.0   
부모자식수        891.0    0.381594    0.806057  0.00    0.0000    0.0000    0.0   
운임요금         891.0   32.204208   49.693429  0.00    7.9104   14.4542   31.0   

                  max  
PassengerId  891.0000  
티켓등급           3.0000  
나이            80.0000  
형제자매배우자수       8.0000  
부모자식수          6.0000  
운임요금         512.3292  


In [63]:
# x_train, y_train을 세로 방향으로 통합한 후, data 변수에 저장하기
data = pd.concat([x_train, y_train], axis = 1)

In [64]:
# '성별' 칼럼에 따라 Survived의 평균값을 구하기
print(data.groupby(['성별'])['Survived'].mean())

성별
female    0.742038
male      0.188908
Name: Survived, dtype: float64


In [65]:
# '티켓등급' 칼럼에 따라 Survived의 평균값을 구하기
print(data.groupby(['티켓등급'])['Survived'].mean())

티켓등급
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


In [66]:
# '선착장' 칼럼에 따라 Survived의 평균값을 구하기
print(data.groupby(['선착장'])['Survived'].mean())

선착장
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64


### 2.2 전처리하기

In [67]:
# 테스트 데이터의 PassengerID 값은 x_test_passenger_id 변수에 저장하기
x_test_passenger_id = x_test['PassengerId']

In [68]:
print(y_train.head())

   PassengerId  Survived
0            1         0
1            2         1
2            3         1
3            4         1
4            5         0


In [69]:
# PassengerId 칼럼을 삭제하기
x_train = x_train.drop(columns = ['PassengerId'])
x_test = x_test.drop(columns = ['PassengerId'])
y_train = y_train.drop(columns = ['PassengerId'])

# 칼럼이 삭제된 상위 5개 행을 확인하기
print(x_train.head())
print(y_train.head())

   티켓등급                                               승객이름      성별    나이  \
0     3                            Braund, Mr. Owen Harris    male  22.0   
1     1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2     3                             Heikkinen, Miss. Laina  female  26.0   
3     1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4     3                           Allen, Mr. William Henry    male  35.0   

   형제자매배우자수  부모자식수              티켓번호     운임요금  객실번호 선착장  
0         1      0         A/5 21171   7.2500   NaN   S  
1         1      0          PC 17599  71.2833   C85   C  
2         0      0  STON/O2. 3101282   7.9250   NaN   S  
3         1      0            113803  53.1000  C123   S  
4         0      0            373450   8.0500   NaN   S  
   Survived
0         0
1         1
2         1
3         1
4         0


In [70]:
# 학습 데이터와 테스트 데이터에서 '티켓번호'칼럼 삭제하기
x_train = x_train.drop(columns = ['티켓번호'])
x_test = x_test.drop(columns = ['티켓번호'])

In [71]:
# 학습데이터와 테스트데이터에서 '승객이름'칼럼 삭제하기
x_train = x_train.drop(columns=['승객이름'])
x_test = x_test.drop(columns=['승객이름'])

In [72]:
# '나이', '객실번호', '선착장' 칼럼의 중복 제외한 값 개수 세기
print(x_train['나이'].isnull().sum())
print(x_train['객실번호'].isnull().sum())
print(x_train['선착장'].isnull().sum())

177
687
2


In [73]:
# '나이'와 'Survived' 칼럼 간의 상관관계를 확인하기
print(data[['나이', 'Survived']].corr())

                나이  Survived
나이        1.000000 -0.077221
Survived -0.077221  1.000000


In [74]:
# x_train에서 '나이' 칼럼을 삭제하기
x_train = x_train.drop(columns = ['나이'])
# x_test에서 '나이' 칼럼을 삭제하기
x_test = x_test.drop(columns = ['나이'])

In [75]:
# x_train의 '객실번호'값에서 중복을 제외한 값의 개수 세기
print(x_train['객실번호'].unique().size)
# x_train에서 '객실번호' 칼럼을 삭제하기
x_train = x_train.drop(columns = ['객실번호'])
# x_test 세트에서 '객실번호' 칼럼을 삭제하기
x_test = x_test.drop(columns = ['객실번호'])

148


In [76]:
# '선착장' 칼럼별로 값의 개수를 세기
print(x_train.groupby(['선착장'])['선착장'].count())

선착장
C    168
Q     77
S    644
Name: 선착장, dtype: int64


In [77]:
print(x_train['선착장'].value_counts())


S    644
C    168
Q     77
Name: 선착장, dtype: int64


In [78]:
# x_train의 '선착장' 칼럼 결측치는 'S' 값으로 적용하기
x_train['선착장'] = x_train['선착장'].fillna('S')
# x_train의 '선착장' 칼럼에 결측치가 있는지 다시 확인하기
print(x_train['선착장'].isnull().sum())

0


In [79]:
# x_test의 '선착장' 칼럼에 결측치가 있는지 다시 확인하기
print(x_test['선착장'].isnull().sum())

0


In [80]:
# 학습 데이터의 '성별' 칼럼을 인코딩하기
x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
# 테스트 데이터의 '성별' 칼럼을 인코딩하기
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)

In [81]:
# x_train의 '선착장'칼럼에 대해 원핫인코딩을 수행한 후, '선착장_dummy'에 저장하기
선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
# 기존 x_train의 우측에 '선착장_dummy' 변수를 덧붙여, x_train에 다시 저장하기
x_train = pd.concat([x_train, 선착장_dummy], axis=1)
# x_train의 상위 5개 행 확인하기
print(x_train.head())

   티켓등급  성별  형제자매배우자수  부모자식수     운임요금 선착장  선착장Q  선착장S
0     3   0         1      0   7.2500   S     0     1
1     1   1         1      0  71.2833   C     0     0
2     3   1         0      0   7.9250   S     0     1
3     1   1         1      0  53.1000   S     0     1
4     3   0         0      0   8.0500   S     0     1


In [82]:
# 학습 데이터에서 '선착장' 칼럼 삭제하기
x_train = x_train.drop(columns=['선착장'])

In [83]:
# x_test 세트의 선착장 칼럼에 대해 원핫인코딩을 수행하고, 기존 칼럼은 삭제하기
선착장_dummy = pd.get_dummies(x_test['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
# 기존 x_test의 우측에 '선착장_dummy'변수를 덧붙여, x_test에 다시 저장하기
x_test = pd.concat([x_test, 선착장_dummy], axis=1)
# 테스트 데이터에서 '선착장' 칼럼 삭제하기
x_test = x_test.drop(columns=['선착장'])

In [84]:
# '형제자매배우자수'와 '부모자식수' 칼럼을 더하여 '가족수'라는 칼럼을 만들기
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
# 결과 확인을 위해, 상위 10개 행을 확인하기
print(x_train[['형제자매배우자수', '부모자식수', '가족수']].head(10))

   형제자매배우자수  부모자식수  가족수
0         1      0    1
1         1      0    1
2         0      0    0
3         1      0    1
4         0      0    0
5         0      0    0
6         0      0    0
7         3      1    4
8         0      2    2
9         1      0    1


In [85]:
# 학습 데이터에서 '형제자매배우자수', '부모자식수' 칼럼 삭제하기
x_train = x_train.drop(columns = ['형제자매배우자수', '부모자식수'])

In [86]:
# '형제자매배우자수'와 '부모자식수'칼럼을 더한 '가족수'라는 칼럼을 만들기
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
# 테스트 데이터에서 '형제자매배우자수', '부모자식수' 칼럼 삭제하기
x_test = x_test.drop(columns=['형제자매배우자수', '부모자식수'])

### 2.3 학습하고 평가하기

In [87]:
# 데이터 분리하기 위한 train_test_split 함수를 가져오기
from sklearn.model_selection import train_test_split
# 학습용과 검증용을 8:2로 분리하여, 각 4개의 변수에 저장하기
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)
# 분리된 데이터의 행/열 구조를 확인하기
print(X_TRAIN.shape)
print(X_TEST.shape)
print(Y_TRAIN.shape)
print(Y_TEST.shape)

(712, 6)
(179, 6)
(712, 1)
(179, 1)


In [88]:
# xgboost 라이브러리에서 제공하는 XGBClassifier 함수를 가져오기
from xgboost import XGBClassifier

In [89]:
# XGB 분류 분석을 수행할 첫번째 모델을 만들고, 공부시키기
model = XGBClassifier(eval_metric = 'error', random_state=10)
model.fit(X_TRAIN, Y_TRAIN)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='error', gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=10,
              reg_alpha=0, reg_lambda=1, ...)

In [90]:
# XGB 분류 분석을 수행할 두 번째 모델을 만들기
model = XGBClassifier(n_estimators = 100, max_depth=5, eval_metric='error', random_state=10)
# 만들어진 모델을 공부시키기
model.fit(X_TRAIN, Y_TRAIN)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='error', gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=10,
              reg_alpha=0, reg_lambda=1, ...)

In [91]:
# 학습이 완료된 모델을 통해 y_test 값을 예측하기: 최종 결과 제출용
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'Survived'})

# y_test_predicted 값 확인하기
print(pd.DataFrame(y_test_predicted).head(10))

   Survived
0         0
1         1
2         0
3         0
4         1
5         0
6         0
7         1
8         1
9         0


In [92]:
# 학습이 완료된 모델을 통해 Y_TEST 값을 예측하기 : 평가지표 계산용
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST))

In [93]:
# sklearn 패키지의 metrics 모듈에서 roc_auc_score 함수를 가져오기
from sklearn.metrics import roc_auc_score

In [94]:
# 1차 학습 모델의 ROC 평가지표 값을 확인하기
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))

0.7983181692859112


In [95]:
# 2차 학습 모델의 ROC 평가지표 값을 확인하기
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))

0.7983181692859112


### 2.4 결과 제출하기

In [96]:
# x_test_passenger_id 변수와 y_test_predicted 변수를 세로 방향으로 붙이기
print(pd.concat([x_test_passenger_id, y_test_predicted], axis=1))

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


In [97]:
# 앞의 출력 결과를 final 변수에 저장하기
final = pd.concat([x_test_passenger_id, y_test_predicted], axis=1)
# final 변수를 data 디렉터리 하위에 12345.csv 이름으로 저장하기
final.to_csv('data/12345.csv', index=False)

In [98]:
# 최종 제출 코드
# 데이터 가져오기
import pandas as pd
x_train = pd.read_csv('bigData-main/titanic_x_train.csv', encoding='ms949')
x_test = pd.read_csv('bigData-main/titanic_x_test.csv', encoding='ms949')
y_train = pd.read_csv('bigData-main/titanic_y_train.csv')
# 전처리하기
x_test_passenger_id = x_test['PassengerId']
x_train = x_train.drop(columns = ['PassengerId'])
x_test = x_test.drop(columns = ['PassengerId'])
y_train = y_train.drop(columns = ['PassengerId'])
x_train = x_train.drop(columns=['티켓번호'])
x_test = x_test.drop(columns = ['티켓번호'])
x_train = x_train.drop(columns=['승객이름'])
x_test = x_test.drop(columns=['승객이름'])
x_train = x_train.drop(columns=['나이'])
x_test = x_test.drop(columns=['나이'])
x_train = x_train.drop(columns=['객실번호'])
x_test = x_test.drop(columns=['객실번호'])
# 결측치 처리하기
x_train['선착장'] = x_train['선착장'].fillna('S')
x_test['선착장'] = x_test['선착장'].fillna('S')

# 인코딩 수행하기
x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)
선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
x_train = pd.concat([x_train, 선착장_dummy], axis=1)
x_train = x_train.drop(columns=['선착장'])
선착장_dummy = pd.get_dummies(x_test['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
x_test = pd.concat([x_test, 선착장_dummy], axis=1)
x_test = x_test.drop(columns=['선착장'])

# 파생변수 만들기
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_train = x_train.drop(columns = ['형제자매배우자수', '부모자식수'])
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
x_test = x_test.drop(columns = ['형제자매배우자수', '부모자식수'])

# 데이터 분리하기
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

# 모델 학습하기
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth=5, eval_metric='error', random_state=10)
model.fit(X_TRAIN, Y_TRAIN)

# 최종 결과를 파일로 저장하기
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'Survived'})
final = pd.concat([x_test_passenger_id, y_test_predicted], axis=1)
final.to_csv('data/12345.csv', index=False)

In [99]:
print(pd.read_csv('data/12345.csv'))

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


## 3. 예측모델 수행하기
고객 10,866건에 대학 학습용 데이터(bike_x_train.csv, bike_y_train.csv)를 이용하여 자전거 대여량 예측 모형을 만든다. 생성한 예측 모형으로 평가용 데이터(bike_x_test.csv)에 해당하는 6,493건의 자전거 대여량 예측값을 다음과 같은 형식의 csv 파일로 생성하시오. 
(제출한 모델의 성능은 R^2 score 평가지표에 따라 채점)

### 3.1 데이터 탐색하기

In [100]:
# pandas 라이브러리 가져오기
import pandas as pd

# 주어진 데이터 파일을 모두 읽어서, 각 데이터 프레임 변수에 저장하기
x_train = pd.read_csv('bigData-main/bike_x_train.csv', encoding='ms949')
x_test = pd.read_csv('bigData-main/bike_x_test.csv', encoding='ms949')
y_train = pd.read_csv('bigData-main/bike_y_train.csv')


In [101]:
# 독립변수인 x_train, x_test의 상위 3개 행을 확인하기
print(x_train.head(3))

          datetime  계절  공휴일  근무일  날씨    온도    체감온도  습도   풍속
0  2011-01-01 0:00   1    0    0   1  9.84  14.395  81  0.0
1  2011-01-01 1:00   1    0    0   1  9.02  13.635  80  0.0
2  2011-01-01 2:00   1    0    0   1  9.02  13.635  80  0.0


In [102]:
print(x_test.head(3))

          datetime  계절  공휴일  근무일  날씨     온도    체감온도  습도       풍속
0  2011-01-20 0:00   1    0    1   1  10.66  11.365  56  26.0027
1  2011-01-20 1:00   1    0    1   1  10.66  13.635  56   0.0000
2  2011-01-20 2:00   1    0    1   1  10.66  13.635  56   0.0000


In [103]:
# 종속변수인 y_train의 상위 3개 행을 확인하기
print(y_train.head(3))

          datetime  count
0  2011-01-01 0:00     16
1  2011-01-01 1:00     40
2  2011-01-01 2:00     32


In [104]:
# 각 데이터 세트의 행과 열 개수를 확인하기
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(10886, 9)
(6493, 9)
(10886, 2)


In [105]:
# x_train 세트의 요약정보 확인하기
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  10886 non-null  object 
 1   계절        10886 non-null  int64  
 2   공휴일       10886 non-null  int64  
 3   근무일       10886 non-null  int64  
 4   날씨        10886 non-null  int64  
 5   온도        10886 non-null  float64
 6   체감온도      10886 non-null  float64
 7   습도        10886 non-null  int64  
 8   풍속        10886 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 765.5+ KB
None


In [106]:
# 범주형 변수들의 값을 확인하기
print(x_train['계절'].unique())
print(x_train['공휴일'].unique())
print(x_train['근무일'].unique())
print(x_train['날씨'].unique())

[1 2 3 4]
[0 1]
[0 1]
[1 2 3 4]


In [107]:
# x_train의 기초 통계량을 확인하고, 가독성을 위해 행/열 바꿔서 출력하기
print(x_train.describe().T)

        count       mean        std   min      25%     50%      75%       max
계절    10886.0   2.506614   1.116174  1.00   2.0000   3.000   4.0000    4.0000
공휴일   10886.0   0.028569   0.166599  0.00   0.0000   0.000   0.0000    1.0000
근무일   10886.0   0.680875   0.466159  0.00   0.0000   1.000   1.0000    1.0000
날씨    10886.0   1.418427   0.633839  1.00   1.0000   1.000   2.0000    4.0000
온도    10886.0  20.230860   7.791590  0.82  13.9400  20.500  26.2400   41.0000
체감온도  10886.0  23.655084   8.474601  0.76  16.6650  24.240  31.0600   45.4550
습도    10886.0  61.886460  19.245033  0.00  47.0000  62.000  77.0000  100.0000
풍속    10886.0  12.799395   8.164537  0.00   7.0015  12.998  16.9979   56.9969


In [108]:
# x_train, y_train을 세로방향으로 통합한 후, data 변수에 저장하기
data = pd.concat([x_train, y_train], axis = 1)

In [109]:
# '계절'칼럼에 따른 count(자전거 대여량)합계를 구하기
# 1(봄), 2(여름), 3(가을), 4(겨울)
print(data.groupby(['계절'])['count'].sum())

계절
1    312498
2    588282
3    640662
4    544034
Name: count, dtype: int64


In [110]:
# '공휴일' 칼럼에 따른 count(자전거 대여량) 합계를 구하기
# 0(공휴일 아님), 1(공휴일)
print(data.groupby(['공휴일'])['count'].sum())

공휴일
0    2027668
1      57808
Name: count, dtype: int64


In [111]:
# '근무일' 칼럼에 따른 count(자전거 대여량) 합계를 구하기
# 0 (근무일 아님), 1 (근무일)
print(data.groupby(['근무일'])['count'].sum())

근무일
0     654872
1    1430604
Name: count, dtype: int64


In [112]:
# '날씨' 칼럼에 따른 count(자전거 대여량) 합계를 구하기
# 1 - 아주 깨끗한 날씨
# 2 - 안개와 구름이 있는 날씨
# 3 - 조금의 눈과 비 또는 조금의 천둥이 치는 날씨
# 4 - 아주 많은 비 또는 우박이 내리는 날씨
print(data.groupby(['날씨'])['count'].sum())

날씨
1    1476063
2     507160
3     102089
4        164
Name: count, dtype: int64


### 3.2 전처리하기

In [114]:
# datetime 칼럼의 데이터 타입을 날짜 타입(datetime)으로 변환하기
x_train['datetime'] = pd.to_datetime(x_train['datetime'])

In [118]:
# x_train의 'datetime' 칼럼에서 연도 데이터를 추출하여 'year' 칼럼에 저장하기
x_train['year'] = x_train['datetime'].dt.year
# x_train의 'year' 칼럼에서 중복 제거한 값을 확인하기
print(x_train['year'].unique())

[2011 2012]


In [125]:
# x_train의 'datetime' 칼럼에서 월 데이터를 추출하여 'month'칼럼에 저장하기
x_train['month'] = x_train['datetime'].dt.month
# x_train의 'month' 칼럼에서 중복 제거한 값을 확인하기
print(x_train['month'].unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12]


In [127]:
# x_train의 'datetime' 칼럼에서 시간 데이터를 추출하여 'day' 칼럼에 저장하기
x_train['day'] = x_train['datetime'].dt.day
# x_train의 'day' 칼럼에서 중복 제거한 값을 확인하기
print(x_train['day'].unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [128]:
# x_train의 'datetime' 칼럼에서 시간 데이터를 추출하여 'hour' 칼럼에 저장하기
x_train['hour'] = x_train['datetime'].dt.hour
# x_train의 'hour' 칼럼에서 중복 제거한 값을 확인하기
print(x_train['hour'].unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [129]:
# x_train의 'datetime'칼럼에서 요일 데이터를 추출하여 'dayofweek' 칼럼에 저장하기
x_train['dayofweek'] = x_train['datetime'].dt.dayofweek
# x_train의 'dayofweek'(요일) 칼럼에서 중복 제거한 값을 확인하기
print(x_train['dayofweek'].unique())

[5 6 0 1 2 3 4]


In [130]:
# 파생 변수가 포함된 독립변수와 종속변수를 통합한 data2 만들기
data2 = pd.concat([x_train, y_train], axis=1)
print(data2.head())

             datetime  계절  공휴일  근무일  날씨    온도    체감온도  습도   풍속  year  month  \
0 2011-01-01 00:00:00   1    0    0   1  9.84  14.395  81  0.0  2011      1   
1 2011-01-01 01:00:00   1    0    0   1  9.02  13.635  80  0.0  2011      1   
2 2011-01-01 02:00:00   1    0    0   1  9.02  13.635  80  0.0  2011      1   
3 2011-01-01 03:00:00   1    0    0   1  9.84  14.395  75  0.0  2011      1   
4 2011-01-01 04:00:00   1    0    0   1  9.84  14.395  75  0.0  2011      1   

   day  hour  dayofweek         datetime  count  
0    1     0          5  2011-01-01 0:00     16  
1    1     1          5  2011-01-01 1:00     40  
2    1     2          5  2011-01-01 2:00     32  
3    1     3          5  2011-01-01 3:00     13  
4    1     4          5  2011-01-01 4:00      1  


In [135]:
# 'year' 칼럼에 따른 'count' 합계를 구하기
print(data2.groupby(['year'])['count'].sum())

year
2011     781979
2012    1303497
Name: count, dtype: int64


In [148]:
# 'month'칼럼에 따른 'count' 합계를 구하기
print(data2.groupby(['month'])['count'].sum())
# x_train에서 'month'칼럼을 삭제하기
x_train = x_train.drop(columns=['month'])

month
1      79884
2      99113
3     133501
4     167402
5     200147
6     220733
7     214617
8     213516
9     212529
10    207434
11    176440
12    160160
Name: count, dtype: int64


In [165]:
# 'day' 칼럼에 따른 'count' 합계를 구하기
print(data2.groupby(['day'])['count'].sum())
# x_train에서 'day' 칼럼을 삭제하기
x_train = x_train.drop(columns='day')

day
1     103692
2     105381
3     111561
4     112335
5     109115
6     108600
7     105486
8     102770
9     108041
10    111645
11    111146
12    109257
13    111448
14    112406
15    115677
16    109837
17    118255
18    108437
19    110387
Name: count, dtype: int64


In [166]:
# 'hour' 칼럼에 따른 'count' 합계를 구하기
print(data2.groupby(['hour'])['count'].sum())

hour
0      25088
1      15372
2      10259
3       5091
4       2832
5       8935
6      34698
7      96968
8     165060
9     100910
10     79667
11     95857
12    116968
13    117551
14    111010
15    115960
16    144266
17    213757
18    196472
19    143767
20    104204
21     79057
22     60911
23     40816
Name: count, dtype: int64


In [167]:
# 'dayofweek' 칼럼에 따른 'count' 합계를 구하기
print(data2.groupby(['dayofweek'])['count'].sum())
# x_train에서 'dayofweek' 칼럼을 삭제하기
x_train = x_train.drop(columns=['dayofweek'])

dayofweek
0    295296
1    291985
2    292226
3    306401
4    302504
5    311518
6    285546
Name: count, dtype: int64


KeyError: "['dayofweek'] not found in axis"

In [ ]:
# 테스트 데이터 세트도 동일하게 적용하기
x_test['datetime'] = pd.to_datetime(x_test['datetime'])
x_test['year'] = x_test['datetime'].dt.year
x_test['hour'] = x_test['datetime'].dt.hour

In [ ]:
# x_test의 datetime 칼럼 값은 x_test_datetime 변수에 저장하기
x_test_datetime = x_test['datetime']
# x_train, x_test에서 datetime 칼럼을 삭제하기
x_train = x_train.drop(columns=['datetime'])
x_test = x_test.drop(columns=['datetime'])
y_train = y_train.drop(columns=['datetime'])

### 3.3 학습하고 평가하기

In [168]:
# 데이터를 분리하기 위한 train_test_split 함수를 가져오기
from sklearn.model_selection import train_test_split
# 학습용과 검증용을 8:2로 분리하여, 각 4개의 변수에 저장하기
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)
# 분리된 데이터의 행/열 구조를 확인하기
print(X_TRAIN.shape)
print(X_TEST.shape)
print(Y_TRAIN.shape)
print(Y_TEST.shape)

(8708, 10)
(2178, 10)
(8708, 1)
(2178, 1)


In [169]:
# xgboost 라이브러리에서 XGBRegressor 함수 가져오기 
from xgboost import XGBRegressor

In [170]:
# 방법 1
# XGB 회귀 분석으로 수행할 첫번째 모델을 만들고, 공부시키기
model = XGBRegressor(n_estimators=100, max_depth=3, random_state=10)
model.fit(X_TRAIN, Y_TRAIN)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=10,
             reg_alpha=0, reg_lambda=1, ...)

In [171]:
print(X_TRAIN.head(3))
print(x_test.head(3))

      계절  공휴일  근무일  날씨     온도    체감온도  습도       풍속  year  hour
7038   2    0    1   3  15.58  19.695  46  15.0013  2012    15
4114   4    0    1   2  16.40  20.455  71  15.0013  2011    11
5720   1    0    1   1   9.84   9.850  38  32.9975  2012    12
   계절  공휴일  근무일  날씨     온도    체감온도  습도       풍속  year  hour
0   1    0    1   1  10.66  11.365  56  26.0027  2011     0
1   1    0    1   1  10.66  13.635  56   0.0000  2011     1
2   1    0    1   1  10.66  13.635  56   0.0000  2011     2


In [172]:
# 학습이 완료된 모델을 통해 y_test 값을 예측하기 : 최종 결과 제출용
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'count'})
# y_test_predicted의 'count' 칼럼 값이 음수인 데이터를 추출하여, 0으로 바꾸기
print(y_test_predicted[y_test_predicted['count']<0])
y_test_predicted[y_test_predicted['count']<0] = 0

# y_test_predicted에서 상위 10개의 행 확인하기
print(pd.DataFrame(y_test_predicted).head(10))

          count
2     -8.262762
3     -9.796791
4     -9.796791
23   -17.563259
26    -2.508549
...         ...
6449  -7.557519
6450 -17.969536
6451 -13.125678
6471  -2.048431
6472  -3.502649

[384 rows x 1 columns]
        count
0    2.139193
1    0.979986
2    0.000000
3    0.000000
4    0.000000
5    7.486256
6   79.849319
7  141.700363
8  319.266785
9   91.251663


In [173]:
# 학습이 완료된 모델을 통해 Y_TEST 값을 예측하기
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST)).rename(columns={0:"count"})
# Y_TEST_PREDICTED의 'count' 칼럼 값이 음수인 데이터를 추출하여, 0dmfh qkRnrl
Y_TEST_PREDICTED[Y_TEST_PREDICTED['count']<0] = 0

In [174]:
# sklearn 패키지의 metrics 모듈에서 r2_score 함수 가져오기
from sklearn.metrics import r2_score

In [175]:
# 1차 학습 모델의 R2평가지표 값을 확인하기 
print(r2_score(Y_TEST, Y_TEST_PREDICTED))

0.9044130641900863


In [177]:
# 두번째 방법
model = XGBRegressor(n_estimators=200, max_depth=5, random_state=10)
model.fit(X_TRAIN, Y_TRAIN)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=200, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=10,
             reg_alpha=0, reg_lambda=1, ...)

In [178]:
# 학습이 완료된 모델을 통해 y_test값을 예측하기 : 최종 결과 제출용
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'count'})

In [180]:
# y_test_predicted의 'count' 칼럼 값이 음수인 데이터를 추출하여, 0으로 바꾸기
y_test_predicted[y_test_predicted['count'] <0] = 0
# y_test_predicted에서 상위 10개의 행 확인하기
print(y_test_predicted.head(10))

        count
0   21.486200
1    0.061608
2    0.000000
3    2.039008
4    2.553952
5   10.101068
6   33.650478
7  109.718521
8  248.734039
9  145.416061


In [182]:
# 학습이 완료된 모델을 통해 Y_TEST 값을 예측하기 : 평가지표 계산용
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST)).rename(columns={0:'count'})
# Y_TEST_PREDICTED의 'count'칼럼이 음수인 데이터를 추출하여, 0으로 바꾸기
Y_TEST_PREDICTED[Y_TEST_PREDICTED['count']<0 ] = 0

In [183]:
# sklearn 패키지의 metrics 모듈에서 r2_score 함수를 가져오기
from sklearn.metrics import r2_score

In [184]:
# 2차 학습 모델의 R2 평가지표 값을 확인하기
print(r2_score(Y_TEST, Y_TEST_PREDICTED))

0.9381182890431057


In [185]:
# x_test_datetime 변수와 y_test_predicted 변수를 세로 방향으로 붙이기
print(pd.concat([x_test_datetime, y_test_predicted], axis=1))

                datetime       count
0    2011-01-20 00:00:00   21.486200
1    2011-01-20 01:00:00    0.061608
2    2011-01-20 02:00:00    0.000000
3    2011-01-20 03:00:00    2.039008
4    2011-01-20 04:00:00    2.553952
...                  ...         ...
6488 2012-12-31 19:00:00  204.254379
6489 2012-12-31 20:00:00  142.610077
6490 2012-12-31 21:00:00  132.809982
6491 2012-12-31 22:00:00   92.404938
6492 2012-12-31 23:00:00   38.964684

[6493 rows x 2 columns]


In [186]:
# 앞의 출력결과를 final 변수에 저자하기
final = pd.concat([x_test_datetime, y_test_predicted], axis=1)
# final 변수를 data 디렉터리 하위에 12345.csv 이름으로 저장하기
final.to_csv('data/12345.csv', index=False)

In [190]:
# 최종 제출 코드
# 데이터 가져오기
import pandas as pd
x_train = pd.read_csv('bigData-main/bike_x_train.csv', encoding='ms949')
x_test = pd.read_csv('bigData-main/bike_x_test.csv', encoding='ms949')
y_train = pd.read_csv('bigData-main/bike_y_train.csv')

# 전처리하기
x_train['datetime'] = pd.to_datetime(x_train['datetime'])
x_train['year'] = x_train['datetime'].dt.year
x_train['hour'] = x_train['datetime'].dt.hour
x_test['datetime'] = pd.to_datetime(x_test['datetime'])
x_test['year'] = x_test['datetime'].dt.year
x_test['hour'] = x_test['datetime'].dt.hour
x_test_datetime = x_test['datetime']
x_train = x_train.drop(columns=['datetime'])
x_test = x_test.drop(columns=['datetime'])
y_train = y_train.drop(columns=['datetime'])

# 데이터 분리하기
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

# 모델을 학습하고 테스트 데이터의 종속변수 값을 예측하기
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=200, max_depth=5, random_state=10)
model.fit(X_TRAIN, Y_TRAIN)
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'count'})
y_test_predicted[y_test_predicted['count'] < 0] = 0
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST)).rename(columns={0:'count'})
Y_TEST_PREDICTED[Y_TEST_PREDICTED['count']<0] = 0

# 모델 평가하기
from sklearn.metrics import r2_score
print(r2_score(Y_TEST, Y_TEST_PREDICTED))

# 결과 제출하기
final = pd.concat([x_test_datetime, y_test_predicted], axis=1)
final.to_csv('data/12345.csv', index=False)

print(pd.read_csv('data/12345.csv'))

0.9381182890431057
                 datetime       count
0     2011-01-20 00:00:00   21.486200
1     2011-01-20 01:00:00    0.061608
2     2011-01-20 02:00:00    0.000000
3     2011-01-20 03:00:00    2.039008
4     2011-01-20 04:00:00    2.553952
...                   ...         ...
6488  2012-12-31 19:00:00  204.254380
6489  2012-12-31 20:00:00  142.610080
6490  2012-12-31 21:00:00  132.809980
6491  2012-12-31 22:00:00   92.404940
6492  2012-12-31 23:00:00   38.964684

[6493 rows x 2 columns]
